In [1]:
import netsquid as ns
import numpy as np
import cmath
import random
import netsquid.components.instructions as instr
from netsquid.components.qprocessor import QuantumProcessor
from netsquid.components.qprocessor import PhysicalInstruction
from netsquid.nodes.connections import Connection, DirectConnection
from netsquid.components import ClassicalChannel
from netsquid.components.models import FibreDelayModel, FixedDelayModel
from netsquid.nodes import Node
from netsquid.components import QuantumChannel
from netsquid.qubits.qubitapi import create_qubits
from netsquid.components.models.qerrormodels import DepolarNoiseModel, DephaseNoiseModel,T1T2NoiseModel

In [2]:

class ClassicalBiConnection(DirectConnection):
    def __init__(self, length,name="ClassicalConnection"):
        
        super().__init__(name=name)
        self.add_subcomponent(ClassicalChannel("Channel_A2B", length=length,
                                               models={"delay_model": FibreDelayModel()}),
                              forward_input=[("A", "send")],
                              forward_output=[("B", "recv")])
        self.add_subcomponent(ClassicalChannel("Channel_B2A", length=length,
                                               models={"delay_model": FibreDelayModel()}),
                              forward_input=[("B", "send")],
                              forward_output=[("A", "recv")])
        
class ClassicalBiConnection_Fix(DirectConnection):
    def __init__(self, length,name="ClassicalConnection"):
        
        super().__init__(name=name)
        self.add_subcomponent(ClassicalChannel("Channel_A2B", length=length,
                                               models={"delay_model": FixedDelayModel(delay = 0)}),
                              forward_input=[("A", "send")],
                              forward_output=[("B", "recv")])
        self.add_subcomponent(ClassicalChannel("Channel_B2A", length=length,
                                               models={"delay_model": FixedDelayModel(delay = 0)}),
                              forward_input=[("B", "send")],
                              forward_output=[("A", "recv")])


class QuantumConnection(Connection):
    def __init__(self, length, prob,name="QuantumConnection"):
        super().__init__(name=name)
        self.prob = prob
        Model = DepolarNoiseModel(depolar_rate = self.prob,time_independent=True)
#         Model = DephaseNoiseModel(dephase_rate  = self.prob,time_independent=True)
        qchannel_a2b = QuantumChannel("qchannel_a2b", length=length,
                                      models={"delay_model": FibreDelayModel(), "quantum_noise_model" : Model})
        # Add channels and forward quantum channel output to external port output:
        self.add_subcomponent(qchannel_a2b,forward_input=[("A","send")],forward_output=[("B", "recv")])
        
class QuantumConnection_Fix(Connection):
    def __init__(self, length, prob,name="QuantumConnection"):
        super().__init__(name=name)
        self.prob = prob
        Model = DepolarNoiseModel(depolar_rate = self.prob,time_independent=True)
#         Model = DephaseNoiseModel(dephase_rate  = self.prob,time_independent=True)
        qchannel_a2b = QuantumChannel("qchannel_a2b", length=length,
                                      models={"delay_model": FixedDelayModel(delay = 0), "quantum_noise_model" : Model})
        # Add channels and forward quantum channel output to external port output:
        self.add_subcomponent(qchannel_a2b,forward_input=[("A","send")],forward_output=[("B", "recv")])
        
# class QuantumConnection(Connection):
#     def __init__(self, length, name="QuantumConnection"):
#         super().__init__(name=name)
#         qchannel_a2b = QuantumChannel("qchannel_a2b", length=length,
#                                       models={"delay_model": FibreDelayModel(), "quantum_noise_model" : T1T2NoiseModel(T1 = 10)})
#         # Add channels and forward quantum channel output to external port output:
#         self.add_subcomponent(qchannel_a2b,forward_input=[("A","send")],forward_output=[("B", "recv")])
        #Connect qsource output to quantum channel input:
#         qsource.ports["qout0"].connect(qchannel_c2a.ports["send"])
#         qsource.ports["qout1"].connect(qchannel_c2b.ports["send"])


In [3]:
def create_processor(num_parties,prob):
    
    num_qubits = int(np.log2(num_parties))
#     print(f"Processor number of qubit: {num_qubits}")
    def UOperator():
        basis_matrix = np.identity(num_parties)
        z = cmath.exp((2*np.pi/num_parties)*1j)
        U = basis_matrix[:,0].reshape(num_parties,1)*np.transpose(basis_matrix[:,0].reshape(num_parties,1))

        i = 1
        while i< num_parties:
            U = U +  z*(basis_matrix[:,i].reshape((num_parties,1))*np.transpose(basis_matrix[:,i].reshape(num_parties,1)))
            i= i+1
        return U

    # K = UOperator(4)
    # print(np.around(K,decimals=2))

    def VOperator():
        basis_matrix = np.identity(num_parties)
        V = np.zeros(num_parties)
        z = cmath.exp((2*np.pi/num_parties)*1j)
        for i in range(num_parties):
            V = V + (z**i)*(basis_matrix[:,i].reshape((num_parties,1))*np.transpose(basis_matrix[:,i].reshape(num_parties,1)))
        return V
    
    def RandUnitary(prob):
        basis_matrix = np.identity(2)
        R= np.zeros(2)
#         Theta = np.random.uniform(0,2*np.pi)
        z = cmath.exp((-prob)*1j)
        R = R + basis_matrix[:,0].reshape((2,1))*np.transpose(basis_matrix[:,0].reshape((2,1))) + z*(basis_matrix[:,1].reshape((2,1))*np.transpose(basis_matrix[:,1].reshape((2,1))))
        return R
    
    R = RandUnitary(prob)
    R1 =  ns.qubits.operators.Operator("R1", R)
    INSTR_R = instr.IGate("R_gate", R1)
    
    
    U1 = UOperator()
    U2 = VOperator()
    R1 =  ns.qubits.operators.Operator("U1", U1)
    R2 =  ns.qubits.operators.Operator("U2", U2)
    INSTR_U = instr.IGate("U_gate", R1)
    INSTR_V = instr.IGate("V_gate", R2)
    
    
#     top = list(range(0,num_qubits))
#     tuple_top = tuple(top)
#     print(f"list of topology{top}")
#     print(f"tuple of topology{tuple_top}")

    # We'll give both Alice and Bob the same kind of processor
    physical_instructions = [
        PhysicalInstruction(instr.INSTR_INIT, duration=3, parallel=True),
        PhysicalInstruction(INSTR_U, duration=1, parallel=True),
        PhysicalInstruction(INSTR_V, duration=1, parallel=True),
        PhysicalInstruction(INSTR_R, duration=1, parallel=True),
        PhysicalInstruction(instr.INSTR_H, duration=1, parallel=True),
        PhysicalInstruction(instr.INSTR_Z, duration=1, parallel=True),
        PhysicalInstruction(instr.INSTR_MEASURE, duration=7, parallel=True),
#         PhysicalInstruction(instr.INSTR_MEASURE, duration=7, parallel=False, topology=[1])
    ]
    processor = QuantumProcessor("quantum_processor", num_positions=num_qubits,phys_instructions=physical_instructions)
    return processor

def create_processor1(num_parties,probs):
    
    num_qubits = int(np.log2(num_parties))
#     print(f"Processor number of qubit: {num_qubits}")
    def UOperator():
        basis_matrix = np.identity(num_parties)
        z = cmath.exp((2*np.pi/num_parties)*1j)
        U = basis_matrix[:,0].reshape(num_parties,1)*np.transpose(basis_matrix[:,0].reshape(num_parties,1))

        i = 1
        while i< num_parties:
            U = U +  z*(basis_matrix[:,i].reshape((num_parties,1))*np.transpose(basis_matrix[:,i].reshape(num_parties,1)))
            i= i+1
        return U

    # K = UOperator(4)
    # print(np.around(K,decimals=2))

    def VOperator():
        basis_matrix = np.identity(num_parties)
        V = np.zeros(num_parties)
        z = cmath.exp((2*np.pi/num_parties)*1j)
        for i in range(num_parties):
            V = V + (z**i)*(basis_matrix[:,i].reshape((num_parties,1))*np.transpose(basis_matrix[:,i].reshape(num_parties,1)))
        return V
    U1 = UOperator()
    U2 = VOperator()
    R1 =  ns.qubits.operators.Operator("U1", U1)
    R2 =  ns.qubits.operators.Operator("U2", U2)
    INSTR_U = instr.IGate("U_gate", R1)
    INSTR_V = instr.IGate("V_gate", R2)
    
    
#     top = list(range(0,num_qubits))
#     tuple_top = tuple(top)
#     print(f"list of topology{top}")
#     print(f"tuple of topology{tuple_top}")

    # We'll give both Alice and Bob the same kind of processor
    physical_instructions = [
        PhysicalInstruction(instr.INSTR_INIT, duration=3, parallel=True),
        PhysicalInstruction(INSTR_U, duration=1, parallel=True),
        PhysicalInstruction(INSTR_V, duration=1, parallel=True),
        PhysicalInstruction(instr.INSTR_H, duration=1, parallel=True),
        PhysicalInstruction(instr.INSTR_Z, duration=1, parallel=True),
        PhysicalInstruction(instr.INSTR_MEASURE, duration=7, parallel=True),
#         PhysicalInstruction(instr.INSTR_MEASURE, duration=7, parallel=False, topology=[1])
    ]
#     memory_noise_model = DepolarNoiseModel(depolar_rate=probs,time_independent = True)
    memory_noise_model = DephaseNoiseModel(dephase_rate  = probs,time_independent=True)
    
    processor = QuantumProcessor("quantum_processor", num_positions=num_qubits,mem_noise_models=memory_noise_model,phys_instructions=physical_instructions)
    return processor

In [4]:
from netsquid.components.qprogram import QuantumProgram

class InitStateProgram(QuantumProgram):
#     default_num_qubits = 4
#     def __init__(self,num_parties)
#         print(num_parties)
#         self.num_qubits_ = int(np.log2(num_parties))
    
    def program(self):
#         self.num_qubits = int(np.log2(self.num_qubits))
        qubits = self.get_qubit_indices()
        self.apply(instr.INSTR_INIT, qubits)
        for i in range(self.num_qubits):   
            self.apply(instr.INSTR_H, qubits[i])
        yield self.run()
        
class UOperate(QuantumProgram):
    
    def UOperator(self,num_parties):
        basis_matrix = np.identity(num_parties)
        z = cmath.exp((2*np.pi/self.num_qubits)*1j)
        U = basis_matrix[:,0].reshape(num_parties,1)*np.transpose(basis_matrix[:,0].reshape(num_parties,1))
        i = 1
        while i< num_parties:
            U = U +  z*(basis_matrix[:,i].reshape((num_parties,1))*np.transpose(basis_matrix[:,i].reshape(num_parties,1)))
            i= i+1
        return U
    
    def program(self):
        
        num_parties = (2**self.num_qubits)
        U1 = self.UOperator(num_parties)  
        R1 =  ns.qubits.operators.Operator("U1", U1)
        INSTR_U = instr.IGate("U_gate",R1)
        qubits = self.get_qubit_indices()
        self.apply(INSTR_U, qubits)
        yield self.run()
        
class VOperate(QuantumProgram):
    
    def VOperator(self,num_parties):
        basis_matrix = np.identity(num_parties)
        V = np.zeros(num_parties)
        z = cmath.exp((2*np.pi/num_parties)*1j)
        for i in range(num_parties):
            V = V + (z**i)*(basis_matrix[:,i].reshape((num_parties,1))*np.transpose(basis_matrix[:,i].reshape(num_parties,1)))
        return V

    def program(self):
        num_parties = (2**self.num_qubits)
        U2 = self.VOperator(num_parties)
        R2 =  ns.qubits.operators.Operator("U2", U2)
        INSTR_V = instr.IGate("V_gate",R2)
        qubits = self.get_qubit_indices()
        self.apply(INSTR_V, qubits)
        yield self.run()
        
class RandUnitary(QuantumProgram):
    def RandUnitary(self,prob):
        basis_matrix = np.identity(2)
        R= np.zeros(2)
#         Theta = np.random.uniform(0,2*np.pi)
        z = cmath.exp((-prob)*1j)
        R = R + basis_matrix[:,0].reshape((2,1))*np.transpose(basis_matrix[:,0].reshape((2,1))) + z*(basis_matrix[:,1].reshape((2,1))*np.transpose(basis_matrix[:,1].reshape((2,1))))
        return R
    
    def program(self,prob):
        qubits = self.get_qubit_indices()
        R = self.RandUnitary(prob)
        R1 =  ns.qubits.operators.Operator("R1", R)
        INSTR_R = instr.IGate("R_gate", R1)
        for i in range(self.num_qubits):   
            self.apply(INSTR_R, qubits[i])

        yield self.run()        
        
class Measure(QuantumProgram):

    def program(self):
        qubits = self.get_qubit_indices()

        for i in range(self.num_qubits):   
            self.apply(instr.INSTR_H, qubits[i])
            self.apply(instr.INSTR_MEASURE, qubits[i], output_key="M"+str(i))
     
        yield self.run()

In [5]:
from netsquid.protocols import NodeProtocol, Signals ,LocalProtocol       

class InitSend(NodeProtocol):
    def __init__(self, node ,name, num_nodes,list_length):
        super().__init__(node, name)
        self.num_nodes = num_nodes
        self.list_length = list_length
        
    def run(self):
#         print(f"Simulation start at {ns.sim_time(ns.MILLISECOND)} ms")
#         print(self.num_nodes)
        qubit_number = int(np.log2(self.num_nodes))# Qubit number is log2 of number of nodes
#Init phase

        #Program to initialize the qubits in the memory, input param: number of qubits
        qubit_init_program = InitStateProgram(num_qubits=qubit_number)
        
        #Program to Apply U Gate, input param: number of qubits
        Uop = UOperate(num_qubits=qubit_number)
        
        #Program to apply V Gate, input param: number of qubits
        Vop = VOperate(num_qubits=qubit_number)
        
        #Indicator variable for case of Initial sending (without waitting port message)
        Initial_send = True
        #Get all port on this node
        #Variable to store classical and quantum ports
        list_port = [k for k in self.node.ports.keys()]
        list_classic = []
        list_quantum = []


        #Put classical ports in list_classic and quantum ports in list_quantum
#         print(list_port)
        for i in range(len(list_port)):
            if (list_port[i][0] == 'c'):
                list_classic.append(list_port[i])
            else:
                list_quantum.append(list_port[i])
#         print(list_classic[-1])
#         print(list_quantum)
        
        for i in range(len(list_quantum)):
            if ((list_quantum[i][1]) == 'o'):
                port_qo = list_quantum[i] #Quantum Input Port
            if ((list_quantum[i][1]) == 'i'):
                port_qi = list_quantum[i] #Quantum Output Port
#         print(self.node.name[1])
        node_num = int(self.node.name.replace('P','')) # Current Node Number    
        
        #Initialize basis count
        basis_sum = 0
        #Initialize loop count for number of state that has been distributed
        k = 0
        
        #Indicator variable for case of valid state (00) measurement
        valid_state = False
        
        #Initialize count for list length
        x = 0
        
# Program Start    
        #Exec init program(create qubits in memory and apply Hadamard gate)
        self.node.qmemory.execute_program(qubit_init_program)
        #Loop For Program
        while True:
#             print(f"Index of program: {k}")
            # Initial send without waitting for port message case (only run once in the beginning)
            if Initial_send:
#                 print("Initial Send")
                # Choose basis and encoding value randomly
                c = random.randint(0,self.num_nodes-1)
                n = random.randint(0,self.num_nodes-1)
#                 c = 0
#                 n = 0
                # Assign current basis summation with current basis
                basis_sum = c
                
                #Wait until qubit is intialized
                yield self.await_program(self.node.qmemory)
                k = k+1

                #Loop to apply U(c) operation
                if c != 0:

                    for i in range(c):
                        yield self.node.qmemory.execute_program(Uop)

                #Loop to apply V(n) operation
                if n != 0:
                    for i in range(n):
                        yield self.node.qmemory.execute_program(Vop)

                
                #Get qubits value from memory
                pos = list(range(0, qubit_number))
                qubits = self.node.qmemory.pop(positions=pos)
                

                #Send qubits quantum output port to next node
                self.node.ports[port_qo].tx_output(qubits) 
        
                #Initial stage finished
                Initial_send = False
            
            # sending after receiving result either from last node or from all nodes
            else:
#                 print("After Init send")
                i = 0
                # Loop to wait for from last node to second node
                while (i<=self.num_nodes-2):
#                     print(list_classic)
#                     print(f"Node {node_num} wait from port {list_classic[-1-i]}")
                    # Wait for node message in input port to the corresponding node
                    yield self.await_port_input(self.node.ports[list_classic[-1-i]])
#                     print(f"Node {node_num} Received from port {list_classic[-1-i]}")
                    # Get message from input node
                    message = self.node.ports[list_classic[-1-i]].rx_input().items[0]
#                     print(message)
                    # Check if last node send valid message or not
                    if (i == 0):
                        if (message == 999):
#                             print("Not valid measurement, reset")
                            # reset basis count
                            basis_sum = 0
                            valid_state = False
                            # Finish loop of waitting
                            break
                    # Check if state is valid, then calculate basis sum
                    if (message != 999):
#                         print("Measurement is valid, calculate basis sum")
                        basis_sum = basis_sum + message
                        valid_state = True
                    # Send basis value to other node (all other node has send basis)
                    if (i == self.num_nodes-2):
#                         print("Send basis to all nodes")
                        for j in range(self.num_nodes-1):
#                             print(f"Node 1 send to port {list_classic[j]}")
                            self.node.ports[list_classic[j]].tx_output(c)
                            
                    i = i+1
                #Record list element if sum of basis mod node_number == 0 and valid state(00 measurement)
                if (basis_sum % self.num_nodes == 0) and valid_state:
#                     print("Record list")
                    global_list[x][0] = n
                    basis_sum = 0
                    x = x+1
#                     if (x > self.list_length-1):
#                         print(f"Number of states = {k}")
#                         print(f"Node {node_num} list distribution ended at: {ns.sim_time(ns.MILLISECOND )} ms")
#                         ns.sim_stop()
#                 print("Init new qubits")
               #Initialize qubits and do gate operation on qubits
                c = random.randint(0,self.num_nodes-1)
                n = random.randint(0,self.num_nodes-1)
#                 c = 0
#                 n = 0
                basis_sum = c
                self.node.qmemory.execute_program(qubit_init_program)
                yield self.await_program(self.node.qmemory)
                if c != 0:
                    for i in range(c):
                        yield self.node.qmemory.execute_program(Uop)
                if n != 0:
                    for i in range(n):
                        yield self.node.qmemory.execute_program(Vop)
                qubits = self.node.qmemory.pop(positions=pos)
                #Send qubits to next port
#                 print("Send qubits to other nodes")
                self.node.ports[port_qo].tx_output(qubits) 
                k = k+1
                
class ReceiveOperate(NodeProtocol):
    def __init__(self, node ,name, num_nodes,list_length):
        super().__init__(node, name)
        self.num_nodes = num_nodes
        self.list_length = list_length
        
    def run(self):

#Init Phase
        qubit_number = int(np.log2(self.num_nodes))
    
        Uop = UOperate(num_qubits=qubit_number)
        Vop = VOperate(num_qubits=qubit_number)
        
        list_port = [k for k in self.node.ports.keys()] # List of Ports of this Node
        list_classic = []# List of classical port
        list_quantum = []# List of quantum port

        for i in range(len(list_port)):
            if (list_port[i][0] == 'c'):
                list_classic.append(list_port[i]) # Put all classical node in the list
            else:
                list_quantum.append(list_port[i])# Put all quantum node in the list
        
#         print(list_classic)
#         print(list_quantum)
        
        for i in range(len(list_quantum)):
            if ((list_quantum[i][1]) == 'o'):
                port_qo = list_quantum[i] #Quantum Input Port
            if ((list_quantum[i][1]) == 'i'):
                port_qi = list_quantum[i] #Quantum Output Port
        

        
#         print(self.node.name[1])
        
        node_num = int(self.node.name.replace('P','')) # Current Node Number
        basis_sum = 0
        k = 0
        valid_state = False
        x = 0
        
#         priv_list = np.ndarray(shape=(100,1),dtype='i')
# Program Phase
        while True:
            c = random.randint(0,self.num_nodes-1)
            n = random.randint(0,1)
            basis_sum = c
#             c = 0
#             n = 0
            yield self.await_port_input(self.node.ports[port_qi])

            if c != 0:
                for i in range(c):
                    yield self.node.qmemory.execute_program(Uop)

            if n == 1:
                yield self.node.qmemory.execute_program(Vop)
            
            # Get all qubits value in the memory
            pos = list(range(0, qubit_number))
            qubits = self.node.qmemory.pop(positions=pos)
            self.node.ports[port_qo].tx_output(qubits)

            i=0
            while (i<=self.num_nodes-2):
#                 print(f"Node {node_num} Loop for basis announcement index: {i}")
#                 print(f"Node {node_num} wait from port {list_classic[-1-i]}")
                yield self.await_port_input(self.node.ports[list_classic[-1-i]])
#                 print(f"Node {node_num} Received from port {list_classic[-1-i]}")
                message = self.node.ports[list_classic[-1-i]].rx_input().items[0]
                if (i == 0):
                    if (message == 999):
                        basis_sum = 0
                        valid_state = False
                        break
                if (message != 999):
                        basis_sum = basis_sum + message
                        valid_state = True
                if (i == (self.num_nodes-1-node_num)):
                    
                    for j in range(self.num_nodes-1):
                        self.node.ports[list_classic[j]].tx_output(c)
#                         print(f"Node {node_num} send to port {list_classic[j]}")

                i= i+1

            if (basis_sum % self.num_nodes == 0) and valid_state:
                global_list[x][node_num-1] = n
                basis_sum = 0
                x = x+1
#                 if (x > self.list_length-1):
#                     print(f"Node {node_num} list distribution ended at: {ns.sim_time(ns.MILLISECOND )} ms")
# #                     ns.sim_stop()
#             k = k+1
class ReceiveOperateMeasure(NodeProtocol):
    def __init__(self, node ,name, num_nodes,list_length):
        super().__init__(node, name)
        self.num_nodes = num_nodes
        self.list_length = list_length

    def run(self):
# Init Phase
        qubit_number = int(np.log2(self.num_nodes))
        
        Uop = UOperate(num_qubits=qubit_number)
        Vop = VOperate(num_qubits=qubit_number)
        measure_program = Measure(num_qubits=qubit_number)
        
        list_port = [k for k in self.node.ports.keys()] # List of Ports of this Node
        list_classic = []# List of classical port
        list_quantum = []# List of quantum port
        
        for i in range(len(list_port)):
            if (list_port[i][0] == 'c'):
                list_classic.append(list_port[i]) # Put all classical node in the list
            else:
                list_quantum.append(list_port[i])# Put all quantum node in the list
#         print(list_classic)
#         print(list_quantum)
        
        for i in range(len(list_quantum)):
            if ((list_quantum[i][1]) == 'o'):
                port_qo = list_quantum[i] #Quantum Input Port
            if ((list_quantum[i][1]) == 'i'):
                port_qi = list_quantum[i] #Quantum Output Port
#         print(self.node.name[1])
        node_num = int(self.node.name.replace('P',''))
        
 
        basis_sum = 0
        k = 0
        x=0
        valid_state = False
        
# Program Phase
        while True:
            c = random.randint(0,self.num_nodes-1)
            n = random.randint(0,1)
#             c = 0
#             n = 0
            basis_sum = c
            k = k+1
            
            yield self.await_port_input(self.node.ports[port_qi])
            
            if c != 0:
                for i in range(c):
                    yield self.node.qmemory.execute_program(Uop)
            if n == 1:
                yield self.node.qmemory.execute_program(Vop)
                
            yield self.node.qmemory.execute_program(measure_program)

            meas = np.ndarray(shape=(qubit_number,1))
            for m in range(qubit_number):
                meas[m] = measure_program.output["M"+str(m)]
#             print(measure_program.output)
#             print(meas)
            if np.all((meas == 0)):
                for i in range(self.num_nodes-1):
                    self.node.ports[list_classic[i]].tx_output(c)
                i=0
                while (i<=self.num_nodes-2):
                    yield self.await_port_input(self.node.ports[list_classic[-1-i]])
                    message = self.node.ports[list_classic[-1-i]].rx_input().items[0]
                    basis_sum = basis_sum + message
                    valid_state = True
                    i = i+1
                    
            else:
                valid_state = False
                basis_sum = 0
                for i in range(self.num_nodes-1):
                    self.node.ports[list_classic[i]].tx_output(999)
                    
            if (basis_sum % self.num_nodes == 0) and valid_state:
                global_list[x][self.num_nodes-1] = n
                basis_sum = 0
                x= x+1
                if (x > self.list_length-1):
#                     print(f"Node {node_num} list distribution ended at: {ns.sim_time(ns.MILLISECOND )} ms")
                    ns.sim_stop()



In [6]:
class ReceiveOperateMeasure1(NodeProtocol):
    def __init__(self, node ,name, prob, num_nodes,list_length):
        super().__init__(node, name)
        self.num_nodes = num_nodes
        self.list_length = list_length
        self.prob = prob
    def run(self):
# Init Phase
        qubit_number = int(np.log2(self.num_nodes))
        
        Uop = UOperate(num_qubits=qubit_number)
        Vop = VOperate(num_qubits=qubit_number)
        measure_program = Measure(num_qubits=qubit_number)
        randU_program = RandUnitary(num_qubits=qubit_number)
        
        list_port = [k for k in self.node.ports.keys()] # List of Ports of this Node
        list_classic = []# List of classical port
        list_quantum = []# List of quantum port
        
        for i in range(len(list_port)):
            if (list_port[i][0] == 'c'):
                list_classic.append(list_port[i]) # Put all classical node in the list
            else:
                list_quantum.append(list_port[i])# Put all quantum node in the list
#         print(list_classic)
#         print(list_quantum)
        
        for i in range(len(list_quantum)):
            if ((list_quantum[i][1]) == 'o'):
                port_qo = list_quantum[i] #Quantum Input Port
            if ((list_quantum[i][1]) == 'i'):
                port_qi = list_quantum[i] #Quantum Output Port
#         print(self.node.name[1])
        node_num = int(self.node.name.replace('P',''))
        
 
        basis_sum = 0
        k = 0
        x=0
        valid_state = False
        
# Program Phase
        while True:
            c = random.randint(0,self.num_nodes-1)
            n = random.randint(0,1)
#             c = 0
#             n = 0
            basis_sum = c
            k = k+1
            
            yield self.await_port_input(self.node.ports[port_qi])
            
            
            if c != 0:
                for i in range(c):
                    yield self.node.qmemory.execute_program(Uop)
            if n == 1:
                yield self.node.qmemory.execute_program(Vop)
                

            yield self.node.qmemory.execute_program(randU_program,prob=self.prob)
            
            
            yield self.node.qmemory.execute_program(measure_program)

            meas = np.ndarray(shape=(qubit_number,1))
            for m in range(qubit_number):
                meas[m] = measure_program.output["M"+str(m)]
#             print(measure_program.output)
#             print(meas)
            if np.all((meas == 0)):
                for i in range(self.num_nodes-1):
                    self.node.ports[list_classic[i]].tx_output(c)
                i=0
                while (i<=self.num_nodes-2):
                    yield self.await_port_input(self.node.ports[list_classic[-1-i]])
                    message = self.node.ports[list_classic[-1-i]].rx_input().items[0]
                    basis_sum = basis_sum + message
                    valid_state = True
                    i = i+1
                    
            else:
                valid_state = False
                basis_sum = 0
                for i in range(self.num_nodes-1):
                    self.node.ports[list_classic[i]].tx_output(999)
                    
            if (basis_sum % self.num_nodes == 0) and valid_state:
                global_list[x][self.num_nodes-1] = n
                basis_sum = 0
                x= x+1
                if (x > self.list_length-1):
                    print(f"Node {node_num} list distribution ended at: {ns.sim_time(ns.MILLISECOND )} ms")
                    ns.sim_stop()


In [7]:
class FaultyInitSend(NodeProtocol):
    def __init__(self, node ,name, num_nodes,list_length):
        super().__init__(node, name)
        self.num_nodes = num_nodes
        self.list_length = list_length
        
    def run(self):
#         print(f"Simulation start at {ns.sim_time(ns.MILLISECOND)} ms")
#         print(self.num_nodes)
        qubit_init_program = InitStateProgram(num_qubits=int(np.log2(self.num_nodes)))
        Uop = UOperate(num_qubits=int(np.log2(self.num_nodes)))
        Vop = VOperate(num_qubits=int(np.log2(self.num_nodes)))
        qubit_initialised = False
        result_received = False
        Initial_send = True
        self.node.qmemory.execute_program(qubit_init_program)
        list_port = [k for k in self.node.ports.keys()]
#         priv_list = np.ndarray(shape=(100,1),dtype='i')
#         print(list_port)
        basis_sum = 0
        k = 0
        valid_state = False
        x = 0
        pos = list(range(0, int(np.log2(self.num_nodes))))
        
        while True:
            print(f"Number of state distributions: {k+1}")
#             print("Node 1 Loop Test")
#             print(f"Number of state: {k+1}")
            if Initial_send:
                c = random.randint(0,self.num_nodes-1)
                n = random.randint(0,self.num_nodes-1)
                r = random.randint(0,self.num_nodes-1)
                basis_sum = r
                yield self.await_program(self.node.qmemory)
#                 print("Start Generate Qubits")
                if c != 0:
#                     print(f"c value: {c}")
                    for i in range(c):
                        yield self.node.qmemory.execute_program(Uop)
#                         print("Node 1 U Operation")
                if n != 0:
                    for i in range(n):
                        yield self.node.qmemory.execute_program(Vop)
#                         print("Node 1 V Operation")
                
                qubits = self.node.qmemory.pop(positions=pos)
#                 print("Node 1 Send Qubits")
                self.node.ports["qo_node_port1"].tx_output(qubits) 
                Initial_send = False
    
            else:
#                 print("Not Init If")
                i = 0
                while (i<=self.num_nodes-2):
                    yield self.await_port_input(self.node.ports[list_port[self.num_nodes-i-2]])
                    message = self.node.ports[list_port[self.num_nodes-i-2]].rx_input().items[0]
#                     print(f"Node 1 Received message from Node {list_port[self.num_nodes-i-2][-1]} {message}")
#                     basis[k][0] = c
                    if (i == 0):
                        if (message == 999):
                            basis_sum = 0
                            valid_state = False
                            break
                    if (message != 999):
                        basis_sum = basis_sum + message
                        valid_state = True
                        
                    if (i == self.num_nodes-2):
                        for j in range(len(list_port)-1):
#                             print(f"Node 1 Send Result to Node {j+2}")
                            self.node.ports[list_port[j]].tx_output(r)
                    i = i+1
#                 if valid_state:
#                     print(f"basis sum: {basis_sum}")
#                     print(f"Node 1 basis: {c}")
#                     print(f"Node 1 list: {n}")
                if (basis_sum % self.num_nodes == 0) and valid_state:
#                     print(f"Node 1 index x:{x} sum: {basis_sum}")
#                     priv_list[x][0] = n
                    global_list[x][0] = n
#                     valid_basis[x][0] = c
                    basis_sum = 0
#                     print(f"valid_basis {x}{0} = {c}")
                    x = x+1
                    if (x > self.list_length-1):
                        print(f"Simulation ended at: {ns.sim_time(ns.MILLISECOND )} ms")
                        ns.sim_stop()
               
                
                c = random.randint(0,self.num_nodes-1)
                n = random.randint(0,self.num_nodes-1)
                r = random.randint(0,self.num_nodes-1)
                basis_sum = r
                self.node.qmemory.execute_program(qubit_init_program)
                yield self.await_program(self.node.qmemory)
#                 print("Start Generate Qubits")
                if c != 0:
#                     print(f"c value: {c}")
                    for i in range(c):
                        yield self.node.qmemory.execute_program(Uop)
#                         print("Node 1 U Operation")
                if n != 0:
                    for i in range(n):
                        yield self.node.qmemory.execute_program(Vop)
#                         print("Node 1 V Operation")
                qubits = self.node.qmemory.pop(positions=pos)
#                 print("Node 1 Send Qubits")
                self.node.ports["qo_node_port1"].tx_output(qubits) 
                k = k+1

class FaultyReceiveOperate(NodeProtocol):
    def __init__(self, node ,name, num_nodes,faulty_node):
        super().__init__(node, name)
        self.num_nodes = num_nodes
        self.faulty_node = faulty_node
    def run(self):
#         print(self.node.name[1])
        Uop = UOperate(num_qubits=int(np.log2(self.num_nodes)))
        Vop = VOperate(num_qubits=int(np.log2(self.num_nodes)))
        list_port = [k for k in self.node.ports.keys()] # List of Ports of this Node
        port = list_port[self.num_nodes-1] # Quantum Input Ports
        string_ = ""
        for i in range(len(self.node.name)-1):
            string_= string_ + self.node.name[i+1]
#         print(int(string_))
        node_num = int(string_) # Node Number
#         print(list_port)
#         print(port)
        basis_sum = 0
        k = 0
        valid_state = False
        x = 0
#         priv_list = np.ndarray(shape=(100,1),dtype='i')
        pos = list(range(0, int(np.log2(self.num_nodes))))
        while True:
            c = random.randint(0,self.num_nodes-1)
            n = random.randint(0,1)
            r = random.randint(0,self.num_nodes-1)
            
#             if int(node_num) == self.faulty_node:
#                 basis_sum = r
#             else:
#                 basis_sum = c
            
            basis_sum = r
#             basis_sum = c
            yield self.await_port_input(self.node.ports[port])
#             print(f"Node {node_num} Received qubit")
            if c != 0:
                for i in range(c):
                    yield self.node.qmemory.execute_program(Uop)
#                     print(f"Node {node_num} U Operation")
            if n == 1:
                yield self.node.qmemory.execute_program(Vop)
#                 print(f"Node {node_num} V Operation")
            qubits = self.node.qmemory.pop(positions=pos)
            self.node.ports[list_port[self.num_nodes]].tx_output(qubits)
#             print(f"Node {node_num} send qubits to Node{int(node_num)+1}")
            
            
            i=0
#             basis[k][int(node_num)-1] = c
            while (i<=self.num_nodes-2):
#                 print(f"Start Loop Index: {node_num}:{i}")
#                 print(f"Node {node_num} waitting port {list_port[self.num_nodes-i-2]}")
                yield self.await_port_input(self.node.ports[list_port[self.num_nodes-i-2]])
                message = self.node.ports[list_port[self.num_nodes-i-2]].rx_input().items[0]
#                 print(f"Node {node_num} Received message from Node {list_port[self.num_nodes-i-2][-1]} {message}")
#                 print(f"index {i} node {node_num}")

                if (i == 0):
                    if (message == 999):
                        basis_sum = 0
                        valid_state = False
                        break
                if (message != 999):

                        basis_sum = basis_sum + message
                        valid_state = True
                if (i == (self.num_nodes-1-int(node_num))):
                    
                    for j in range(self.num_nodes-1):
#                         print(f"Node {node_num} Send Result to Node {list_port[j][-1]}")
#                         self.node.ports[list_port[j]].tx_output(c)
                        self.node.ports[list_port[j]].tx_output(r)

#                         if int(node_num)==self.faulty_node:
#                             self.node.ports[list_port[j]].tx_output(r)
#                         else:
#                             self.node.ports[list_port[j]].tx_output(c)
                i= i+1
#             if valid_state:
#                 print(f"basis sum: {basis_sum}")
#                 print(f"Node {node_num} basis: {c}")
#                 print(f"Node {node_num} list: {n}")
            if (basis_sum % self.num_nodes == 0) and valid_state:
#                 print(f"Node {node_num} x:{x} sum: {basis_sum}")
#                 priv_list[x][0] = n
                global_list[x][int(node_num)-1] = n
#                 valid_basis[x][int(node_num)-1] = c
                basis_sum = 0
                x = x+1
#                 if (x > 99):
#                     print(f"Simulation ended at: {ns.sim_time(ns.MILLISECOND )} ms")
#                     ns.sim_stop()
            k = k+1                

class FaultyReceiveOperateMeasure1(NodeProtocol):
    def __init__(self, node ,name, num_nodes):
        super().__init__(node, name)
        self.num_nodes = num_nodes
        
    def run(self):
        
        Uop = UOperate(num_qubits=int(np.log2(self.num_nodes)))
        Vop = VOperate(num_qubits=int(np.log2(self.num_nodes)))
        measure_program = Measure(num_qubits=int(np.log2(self.num_nodes)))
        list_port = [k for k in self.node.ports.keys()]
        port = list_port[self.num_nodes-1]
        node_num = self.num_nodes
#         print(list_port)
        basis_sum = 0
        k = 0
#         priv_list = np.ndarray(shape=(500,1),dtype='i')
        x=0
        valid_state = False
        pos = list(range(0, int(np.log2(self.num_nodes))))
        while True:
#             random.seed(0)
            c = random.randint(0,self.num_nodes-1)
            n = random.randint(0,1)
            basis_sum = c
#             basis[k][3] = c
            k = k+1
            yield self.await_port_input(self.node.ports[port])
#             print(self.node.ports[self.node.ports["qin_port_node2"].rx_input()])
#             print(f"Node {node_num} Received qubit")
#           message = self.node.ports["qin_port_node2"].rx_input().items
            if c != 0:
                for i in range(c):
                    yield self.node.qmemory.execute_program(Uop)
#                     print(f"Node {node_num} U Operation")
#           yield self.await_program(self.node.qmemory)
            if n == 1:
                yield self.node.qmemory.execute_program(Vop)
#                 print(f"Node {node_num} V Operation")
#           yield self.await_program(self.node.qmemory)
            yield self.node.qmemory.execute_program(measure_program)
#             print(f"Node {node_num} Measure Operation")
            meas = np.ndarray(shape=(int(np.log2(self.num_nodes)),1))
            for m in range(int(np.log2(self.num_nodes))):
                meas[m] = measure_program.output["M"+str(m)] 
#             m1, = measure_program.output["M1"]
#             m2, = measure_program.output["M2"]
#             print("Measurement result: ")
#             print(meas)

            z = np.random.binomial(1, 0.7)
            if (z == 0):
                if not (np.all((meas == 0))):
                    for i in range(len(list_port)-1):
    #                     print(f"Node {node_num} Send Result to Node {i+1}")
                        self.node.ports[list_port[i]].tx_output(c)
                    i=0
                    while (i<=self.num_nodes-2):
        #                 print(f"Node 4 awaitting msg from {list_port[2-i]}")
                        yield self.await_port_input(self.node.ports[list_port[self.num_nodes-i-2]])
                        message = self.node.ports[list_port[self.num_nodes-i-2]].rx_input().items[0]
                        basis_sum = basis_sum + message
                        valid_state = True
    #                     print(f"Node {node_num} Received message from Node {list_port[self.num_nodes-i-2][-1]} {message}")
                        i = i+1

                else:
                    valid_state = False
                    basis_sum = 0
                    for i in range(len(list_port)-1):
    #                     print(f"Node {node_num} Send Discard to Node {i+1}")
                        self.node.ports[list_port[i]].tx_output(999)
    #                     print(f"send through port {list_port[i]}")
    #             if valid_state:
    #                 print(f"basis sum: {basis_sum}")
    #                 print(f"Node 4 basis: {c}")
    #                 print(f"Node 4 list: {n}")                    
                if (basis_sum % self.num_nodes == 0) and valid_state:
    #                 print(f"Node 4 x:{x} sum: {basis_sum}")
    #                 priv_list[x][0] = n
                    global_list[x][self.num_nodes-1] = n
    #                 valid_basis[x][self.num_nodes-1] = c
                    basis_sum = 0
                    x= x+1
            else:
                if (np.all((meas == 0))):
                    for i in range(len(list_port)-1):
    #                     print(f"Node {node_num} Send Result to Node {i+1}")
                        self.node.ports[list_port[i]].tx_output(c)
                    i=0
                    while (i<=self.num_nodes-2):
        #                 print(f"Node 4 awaitting msg from {list_port[2-i]}")
                        yield self.await_port_input(self.node.ports[list_port[self.num_nodes-i-2]])
                        message = self.node.ports[list_port[self.num_nodes-i-2]].rx_input().items[0]
                        basis_sum = basis_sum + message
                        valid_state = True
    #                     print(f"Node {node_num} Received message from Node {list_port[self.num_nodes-i-2][-1]} {message}")
                        i = i+1

                else:
                    valid_state = False
                    basis_sum = 0
                    for i in range(len(list_port)-1):
    #                     print(f"Node {node_num} Send Discard to Node {i+1}")
                        self.node.ports[list_port[i]].tx_output(999)
    #                     print(f"send through port {list_port[i]}")
    #             if valid_state:
    #                 print(f"basis sum: {basis_sum}")
    #                 print(f"Node 4 basis: {c}")
    #                 print(f"Node 4 list: {n}")                    
                if (basis_sum % self.num_nodes == 0) and valid_state:
    #                 print(f"Node 4 x:{x} sum: {basis_sum}")
    #                 priv_list[x][0] = n
                    global_list[x][self.num_nodes-1] = n
    #                 valid_basis[x][self.num_nodes-1] = c
                    basis_sum = 0
                    x= x+1
                else:
                    basis_sum = 0
#                 if (x > 99):
#                     print(f"Simulation ended at: {ns.sim_time(ns.MILLISECOND )} ms")
#                     ns.sim_stop()

class FaultyReceiveOperateMeasure2(NodeProtocol):
    def __init__(self, node ,name, num_nodes):
        super().__init__(node, name)
        self.num_nodes = num_nodes
        
    def run(self):
        
        Uop = UOperate(num_qubits=int(np.log2(self.num_nodes)))
        Vop = VOperate(num_qubits=int(np.log2(self.num_nodes)))
        measure_program = Measure(num_qubits=int(np.log2(self.num_nodes)))
        list_port = [k for k in self.node.ports.keys()]
        port = list_port[self.num_nodes-1]
        node_num = self.num_nodes
#         print(list_port)
        basis_sum = 0
        k = 0
#         priv_list = np.ndarray(shape=(500,1),dtype='i')
        x=0
        valid_state = False
        pos = list(range(0, int(np.log2(self.num_nodes))))
        while True:
#             random.seed(0)
            c = random.randint(0,self.num_nodes-1)
            n = random.randint(0,1)
            r = random.randint(0,self.num_nodes-1)
            basis_sum = r
#             basis[k][3] = c
            k = k+1
            yield self.await_port_input(self.node.ports[port])
#             print(self.node.ports[self.node.ports["qin_port_node2"].rx_input()])
#             print(f"Node {node_num} Received qubit")
#           message = self.node.ports["qin_port_node2"].rx_input().items
            if c != 0:
                for i in range(c):
                    yield self.node.qmemory.execute_program(Uop)
#                     print(f"Node {node_num} U Operation")
#           yield self.await_program(self.node.qmemory)
            if n == 1:
                yield self.node.qmemory.execute_program(Vop)
#                 print(f"Node {node_num} V Operation")
#           yield self.await_program(self.node.qmemory)
            yield self.node.qmemory.execute_program(measure_program)
#             print(f"Node {node_num} Measure Operation")
            meas = np.ndarray(shape=(int(np.log2(self.num_nodes)),1))
            for m in range(int(np.log2(self.num_nodes))):
                meas[m] = measure_program.output["M"+str(m)]
            print(measure_program.output)    
#             m1, = measure_program.output["M1"]

#             m2, = measure_program.output["M2"]
#             print("Measurement result: ")
#             print(meas)
                
            if np.all((meas == 0)):
                for i in range(len(list_port)-1):
#                     print(f"Node {node_num} Send Result to Node {i+1}")
                    self.node.ports[list_port[i]].tx_output(r)
                i=0
                while (i<=self.num_nodes-2):
    #                 print(f"Node 4 awaitting msg from {list_port[2-i]}")
                    yield self.await_port_input(self.node.ports[list_port[self.num_nodes-i-2]])
                    message = self.node.ports[list_port[self.num_nodes-i-2]].rx_input().items[0]
                    basis_sum = basis_sum + message
                    valid_state = True
#                     print(f"Node {node_num} Received message from Node {list_port[self.num_nodes-i-2][-1]} {message}")
                    i = i+1
                    
            else:
                valid_state = False
                basis_sum = 0
                for i in range(len(list_port)-1):
#                     print(f"Node {node_num} Send Discard to Node {i+1}")
                    self.node.ports[list_port[i]].tx_output(999)
#                     print(f"send through port {list_port[i]}")
#             if valid_state:
#                 print(f"basis sum: {basis_sum}")
#                 print(f"Node 4 basis: {c}")
#                 print(f"Node 4 list: {n}")                    
            if (basis_sum % self.num_nodes == 0) and valid_state:
#                 print(f"Node 4 x:{x} sum: {basis_sum}")
#                 priv_list[x][0] = n
                global_list[x][self.num_nodes-1] = n
#                 valid_basis[x][self.num_nodes-1] = c
                basis_sum = 0
                x= x+1
#                 if (x > 99):
#                     print(f"Simulation ended at: {ns.sim_time(ns.MILLISECOND )} ms")
#                     ns.sim_stop()

In [8]:
from netsquid.nodes import Network
def example_network_setup(num_nodes,prob,node_distance=4e-3):
#     print("Network Setup")
    nodes =[]
    i = 1
    while i<=(num_nodes):
        if i == 4:
            nodes.append(Node(f"P{i}",qmemory = create_processor1(num_nodes,prob)))
#             nodes.append(Node(f"P{i}",qmemory = create_processor(num_nodes,prob)))
        else:
            nodes.append(Node(f"P{i}",qmemory = create_processor(num_nodes,prob)))
#         nodes.append(Node(f"P{i}",qmemory = create_processor1(num_nodes,prob)))
        i= i+1
    
    # Create a network
    network = Network("List Distribution Network")
#     print(nodes)
    network.add_nodes(nodes)
#     print("Nodes completed")
    
    i = 1
    while i< (num_nodes):
        node = nodes[i-1]
        j = 1
        while j<=(num_nodes-i):
            node_next = nodes[i+j-1]
            if ((i-1) == 0) and ((i+j-1) == 1):
#                 c_conn = ClassicalBiConnection(name =f"c_conn{i}{i+j}", length = node_distance)
                c_conn = ClassicalBiConnection_Fix(name =f"c_conn{i}{i+j}", length = node_distance)
            else:
                c_conn = ClassicalBiConnection(name =f"c_conn{i}{i+j}", length = node_distance)
            network.add_connection(node,node_next, connection= c_conn, label="classical", 
                                   port_name_node1 = f"cio_node_port{i}{i+j}", port_name_node2 = f"cio_node_port{i+j}{i}")
            
            j = j+1
        i = i+1
#     print("Classical Conn Completed")

    i =1
    while i<(num_nodes):
#         print(i)
        node, node_right = nodes[i-1], nodes[i]
#         q_conn = QuantumConnection(name=f"qconn_{i}{i+1}", length=node_distance,prob=prob)
        if i == 1 :
            q_conn = QuantumConnection_Fix(name=f"qconn_{i}{i+1}", length=node_distance,prob=0)
#             q_conn = QuantumConnection(name=f"qconn_{i}{i+1}", length=node_distance,prob=0)
        else:
            q_conn = QuantumConnection(name=f"qconn_{i}{i+1}", length=node_distance,prob=0)
        network.add_connection(node, node_right, connection=q_conn, label="quantum", port_name_node1 = f"qo_node_port{i}", port_name_node2=f"qin_node_port{i+1}")
        i= i+1
#     print("Quantum Conn Completed")

    i = 2
    while i<=(num_nodes):
        nodes[i-1].ports[f"qin_node_port{i}"].forward_input(nodes[i-1].qmemory.ports['qin'])
        i = i+1
#     print("End Network Setup")
    
    return network

In [9]:
def setup_protocol(network,nodes_num,prob,fault_num,list_length):
#     print("Setup Protocol")
    
    protocol = LocalProtocol(nodes=network.nodes)
    nodes = []
    i = 1
    while i<=(nodes_num):
        nodes.append(network.get_node("P"+str(i)))
        i = i+1
#     print(nodes)

    
    subprotocol = InitSend(node=nodes[0],name=f"Init_Send{nodes[0].name}",num_nodes=nodes_num,list_length=list_length)
#     subprotocol = FaultyInitSend(node=nodes[0],name=f"Faulty Init_Send{nodes[0].name}",num_nodes=nodes_num,list_length=list_length)
    protocol.add_subprotocol(subprotocol)

    i=1
    j = 0
    while i<= (nodes_num-2):
        if j<fault_num:
            subprotocol = FaultyReceiveOperate(node=nodes[i], name=f"FaultyReceive_Operate{nodes[i].name}",num_nodes=nodes_num,faulty_node=i)
            protocol.add_subprotocol(subprotocol)
        else:
            subprotocol = ReceiveOperate(node=nodes[i], name=f"Receive_Operate{nodes[i].name}",num_nodes=nodes_num,list_length=list_length)
            protocol.add_subprotocol(subprotocol)
        j = j+1
        i = i+1
    subprotocol = ReceiveOperateMeasure(node=nodes[nodes_num-1],name=f"Receive_Operate_Send{nodes[nodes_num-1].name}",num_nodes=nodes_num,list_length=list_length)
#     subprotocol = ReceiveOperateMeasure1(node=nodes[nodes_num-1],name=f"Receive_Operate_Send{nodes[nodes_num-1].name}",prob=prob,num_nodes=nodes_num,list_length=list_length)
#     subprotocol = FaultyReceiveOperateMeasure1(node=nodes[nodes_num-1],name=f"Receive_Operate_Send{nodes[nodes_num-1].name}",num_nodes=nodes_num)
#     subprotocol = FaultyReceiveOperateMeasure2(node=nodes[nodes_num-1],name=f"Faulty Receive_Operate_Send{nodes[nodes_num-1].name}",num_nodes=nodes_num)
    protocol.add_subprotocol(subprotocol)
#     print("End Setup Protocol")
    return protocol


In [ ]:
from netsquid.util.simtools import set_random_state
import pandas as pd
# set up initial parameters
nodes_num = 4 #Node numbers
fault_num = 0 #Faulty node numbers
# exp_number = 1 #Experiment numbers
probs = np.linspace(0, 1, num=100)


# probs = np.linspace(0,2*np.pi, num=3)


exp_number = len(probs)
list_length = 100 #List length
error_array = np.ndarray(shape=(exp_number,2))

x=0
basis = np.ndarray(shape=(list_length,nodes_num), dtype='i')
global_list = np.ndarray(shape=(list_length,nodes_num), dtype='i')
average = 100
while x < 50:
    error_sum = 0
    for z in range (average):
        ns.sim_reset()
        network = example_network_setup(nodes_num,probs[x],node_distance=4)
    #     protocol = setup_protocol(network,nodes_num,fault_num,list_length)
        protocol = setup_protocol(network,nodes_num,probs[x],fault_num,list_length)
        protocol.start()
        stats = ns.sim_run()
        if (ns.sim_state() == 2):
            valid_sum = 0
            for i in range(global_list.shape[0]-1):
                row_sum = 0
                for j in range (global_list.shape[1]):
                    row_sum = row_sum + global_list[i][j]
                if ((row_sum % nodes_num) == 0):
                    valid_sum = valid_sum+1
    #         percentage_correct = (valid_sum/(global_list.shape[0]-1)) * 100
            percentage_correct = (valid_sum/(global_list.shape[0]-1))
            error_ratio = 1-percentage_correct
            error_sum = error_sum + error_ratio
    print(f"Probs Value = {probs[x]} Averaged Ratio of Error List:{error_sum/average}")
#     print(global_list)
#         print(f"Percentage of Correct List: {round(percentage_correct,3)}%")
    error_array[x][0] = probs[x]
    error_array[x][1] = error_sum/average
    x = x+1



In [ ]:
# print(global_list)
# print(error_array)

In [ ]:
print(error_array)
error_data = pd.DataFrame(data = error_array,columns = ['error probability','Error List Ratio'])
print(error_data)
error_data.to_csv('Qudit_DephaseLocal_3party_1_LastParty.csv')

In [ ]:
# print(global_list)

sum_0 = 0
sum_1 = 0
sum_2 = 0
sum_3 = 0
print(global_list.shape)
for i in range(global_list.shape[0]):
    if global_list[i][0] == 0:
        sum_0 = sum_0 +1
    elif(global_list[i][0] == 1):
        sum_1 = sum_1+1
    elif(global_list[i][0] == 2):
        sum_2 = sum_2 +1
    else:
        sum_3 = sum_3 +1
print(f"Prob of 0 = {sum_0/global_list.shape[0]}")
print(f"Prob of 1 = {sum_1/global_list.shape[0]}")
print(f"Prob of 2 = {sum_2/global_list.shape[0]}")
print(f"Prob of 3 = {sum_3/global_list.shape[0]}")

